In [1]:
!pip install ipython-autotime

%load_ext autotime

time: 2.11 ms (started: 2021-08-28 20:31:53 +00:00)


# Instala a lib Reranker

In [2]:
%cd /content/drive/MyDrive/__TCC__Reranker/Reranker

/content/drive/MyDrive/__TCC__Reranker/Reranker
time: 4.53 ms (started: 2021-08-28 20:31:54 +00:00)


In [3]:
%pip install -e .

Obtaining file:///content/drive/MyDrive/__TCC__Reranker/Reranker
     |████████████████████████████████| 2.6 MB 8.1 MB/s 
     |████████████████████████████████| 264 kB 70.1 MB/s 
     |████████████████████████████████| 118 kB 72.4 MB/s 
     |████████████████████████████████| 243 kB 69.9 MB/s 
     |████████████████████████████████| 50 kB 8.3 MB/s 
     |████████████████████████████████| 3.3 MB 58.9 MB/s 
     |████████████████████████████████| 636 kB 56.9 MB/s 
     |████████████████████████████████| 895 kB 79.0 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
  Running setup.py develop for reranker
time: 15.1 s (started: 2021-08-28 20:31:54 +00:00)


In [4]:
!export HF_DATASETS_CACHE="/content/drive/My Drive/__TCC__Reranker"

time: 109 ms (started: 2021-08-28 20:32:10 +00:00)


## Roda todos os arquivos do HDCT

ToDo: colocar esses comandos num .sh
* essa parte não precisa de GPU


In [5]:
%cd run_msmarco

/content/drive/My Drive/__TCC__Reranker/Reranker/run_msmarco
time: 3.48 ms (started: 2021-08-28 20:32:10 +00:00)


# Preprocessa apenas 1 arquivo hdct

In [6]:
!python /content/drive/MyDrive/__TCC__Reranker/Reranker/run_msmarco/scripts/1_preprocess.py \
    --tokenizer_name bert-base-uncased \
    --rank_file 	/content/drive/MyDrive/__TCC__Reranker/Reranker/common_data/01-hdct-marco-train/000.txt \
    --json_dir /content/drive/MyDrive/__TCC__Reranker/Reranker/run_msmarco/data/training_files \
    --n_sample 10 \
    --sample_from_top 100 \
    --random \
    --truncate 512 \
    --qrel /content/drive/MyDrive/__TCC__Reranker/Reranker/common_data/02-msmarco-files/msmarco-doctrain-qrels.tsv.gz \
    --query_collection /content/drive/MyDrive/__TCC__Reranker/Reranker/common_data/02-msmarco-files/msmarco-doctrain-queries.tsv \
    --doc_collection /content/drive/MyDrive/__TCC__Reranker/Reranker/common_data/02-msmarco-files/msmarco-docs.tsv


======== NOVO DE NOVO =========

0 queries not judged and skipped
Using custom data configuration default-336b90812e3158ed
Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-336b90812e3158ed/0.0.0/9144e0a4e8435090117cea53e6c7537173ef2304525df4a077c435d8ee7828ff. Subsequent calls will reuse this data.
Using custom data configuration default-81f490248c7c9f05
Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-81f490248c7c9f05/0.0.0/9144e0a4e8435090117cea53e6c7537173ef2304525df4a077c435d8ee7828ff. Subsequent calls will reuse this data.
Downloading: 100% 28.0/28.0 [00:00<00:00, 28.9kB/s]
Downloading: 100% 570/570 [00:00<00:00, 602kB/s]
Downloading: 100% 232k/232k [00:00<00:00, 1.21MB/s]
Downloading: 100% 466k/466k [00:00<00:00, 1.58MB/s]
  0% 0/2000 [00:00<?, ?it/s]======= Vamos encodar os negatives ========
[8299, 1024, 1013, 1013, 7479, 1012, 2381, 1012, 4012, 1013, 7832, 1013, 2088, 1011, 2162, 1011, 2462, 1013, 8647, 1

# Realiza treino

In [7]:
!python -m torch.distributed.launch \
  --nproc_per_node 1 /content/drive/MyDrive/__TCC__Reranker/Reranker/common_scripts/2_run_marco.py \
  --output_dir /content/drive/MyDrive/__TCC__Reranker/Reranker/run_msmarco/data/checkpoints \
  --model_name_or_path  bert-base-uncased \
  --do_train \
  --save_steps 2000 \
  --train_dir /content/drive/MyDrive/__TCC__Reranker/Reranker/run_msmarco/data/training_files/ \
  --max_len 512 \
  --fp16 \
  --per_device_train_batch_size 1 \
  --train_group_size 8 \
  --gradient_accumulation_steps 1 \
  --per_device_eval_batch_size 64 \
  --warmup_ratio 0.1 \
  --weight_decay 0.01 \
  --learning_rate 1e-5 \
  --num_train_epochs 2 \
  --overwrite_output_dir \
  --dataloader_num_workers 8 \

/usr/local/lib/python3.7/dist-packages/torch/distributed/launch.py:164: DeprecationWarning: The 'warn' method is deprecated, use 'warning' instead
  "The module torch.distributed.launch is deprecated "
The module torch.distributed.launch is deprecated and going to be removed in future.Migrate to torch.distributed.run
 Please read local_rank from `os.environ('LOCAL_RANK')` instead.
INFO:torch.distributed.launcher.api:Starting elastic_operator with launch configs:
  entrypoint       : /content/drive/MyDrive/__TCC__Reranker/Reranker/common_scripts/2_run_marco.py
  min_nodes        : 1
  max_nodes        : 1
  nproc_per_node   : 1
  run_id           : none
  rdzv_backend     : static
  rdzv_endpoint    : 127.0.0.1:29500
  rdzv_configs     : {'rank': 0, 'timeout': 900}
  max_restarts     : 3
  monitor_interval : 5
  log_dir          : None
  metrics_cfg      : {}

INFO:torch.distributed.elastic.agent.server.local_elastic_agent:log directory set to: /tmp/torchelastic_36n2fdhu/none_9car5_fl
I

# Realiza Inference

In [8]:
!python /content/drive/MyDrive/__TCC__Reranker/Reranker/common_scripts/2_run_marco.py \
  --model_name_or_path /content/drive/MyDrive/__TCC__Reranker/Reranker/run_msmarco/data/checkpoints \
  --output_dir /content/drive/MyDrive/__TCC__Reranker/Reranker/run_msmarco/data/eval \
  --tokenizer_name bert-base-uncased \
  --do_predict \
  --max_len 512 \
  --fp16 \
  --per_device_eval_batch_size 64 \
  --dataloader_num_workers 8 \
  --pred_path /content/drive/MyDrive/__TCC__Reranker/Reranker/common_data/04-inference_files/all.json \
  --pred_id_file  /content/drive/MyDrive/__TCC__Reranker/Reranker/common_data/04-inference_files/ids.tsv \
  --rank_score_path /content/drive/MyDrive/__TCC__Reranker/Reranker/run_msmarco/data/eval/score.txt



AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAa

=========== CONFIGURANDO AS COISAS ===========
file /content/drive/MyDrive/__TCC__Reranker/Reranker/run_msmarco/data/checkpoints/config.json not found
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/transformers/configuration_utils.py", line 520, in get_config_dict
    user_agent=user_agent,
  File "/usr/local/lib/python3.7/dist-packages/transformers/file_utils.py", line 1385, in cached_path
    raise EnvironmentError(f"file {url_or_filename} not found")
OSError: file /content/drive/MyDrive/__TCC__Reranker/Reranker/run_msmarco/data/checkpoints/config.json not found

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/content/drive/MyDrive/__TCC__Reranker/Reranker/common_scripts/2_run_marco.py", line 192, in <module>
    main()
  File "/content/drive/MyDrive/__TCC__Reranker/Reranker/common_scripts/2_run_marco.py",